In [1]:
from utils import *
import gensim
from gensim.models import Word2Vec
from tqdm import tqdm_notebook as tqdm

In [2]:
data_dir = '/datadrive/data_cs/'

In [3]:
cite_dict = defaultdict(lambda: 0)
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PR_CS_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/PR_CS_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        cite_dict[l[1]] += 1

In [10]:
data_dir = '/datadrive/data_med/'
graph = dill.load(open(data_dir + 'graph.pk', 'rb'))

In [4]:
pfl = defaultdict(lambda: {})
num_lines = sum(1 for line in open('/datadrive/MAG_0919/Papers_CS_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/Papers_CS_20190919.tsv') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        bound = min(2020 - int(l[1]), 20)
        if cite_dict[l[0]] < bound or l[0] == '' or l[1] == '' or l[2] == '' or l[3] == '' and l[4] == '' or int(l[1]) < 1900:
            continue
        pi = {'id': l[0], 'title': l[2], 'type': 'paper', 'time': int(l[1])}
        pfl[l[0]] = pi

In [5]:
from pytorch_transformers import *
device = torch.device("cuda:2")
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased',
                                    output_hidden_states=True,
                                    output_attentions=True).to(device)

In [6]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PAb_CS_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/PAb_CS_20190919.tsv') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        try:
            l = l.split('\t')
            if l[0] in pfl:
                input_ids = torch.tensor([tokenizer.encode(pfl[l[0]]['title'])]).to(device)[:, :64]
                if len(input_ids[0]) < 4:
                    continue
                all_hidden_states, all_attentions = model(input_ids)[-2:]
                rep = (all_hidden_states[-2][0] * all_attentions[-2][0].mean(dim=0).mean(dim=0).view(-1, 1)).sum(dim=0)
                pfl[l[0]]['emb'] = rep.tolist()
        except Exception as e:
            print(e)

In [230]:
vfi_ids = {}
num_lines = sum(1 for line in open('/datadrive/MAG_0919/vfi_vector.tsv', 'r'))
with open('/datadrive/MAG_0919/vfi_vector.tsv', 'r') as fin:
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        vfi_ids[l[0]] = True

In [231]:
graph = Graph()
rem = []
num_lines = sum(1 for line in open('/datadrive/MAG_0919/Papers_CS_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/Papers_CS_20190919.tsv') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] not in pfl or l[4] != 'en' or 'emb' not in pfl[l[0]] or l[3] not in vfi_ids:
            continue
        rem += [l[0]]
        vi = {'id': l[3], 'type': 'venue', 'attr': l[-2]}
        graph.add_edge(pfl[l[0]], vi, time = int(l[1]), relation_type = 'PV_' + l[-2])
pfl = {i: pfl[i] for i in rem}
print(len(pfl))

544244


In [232]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PR_CS_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/PR_CS_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] in pfl and l[1] in pfl:
            p1 = pfl[l[0]]
            p2 = pfl[l[1]]
            if p1['time'] >= p2['time']:
                graph.add_edge(p1, p2, time = p1['time'], relation_type = 'PP_cite')

In [233]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PF_CS_20190919.tsv', 'r'))
ffl = {}
with open('/datadrive/MAG_0919/PF_CS_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] in pfl and l[1] in vfi_ids:
            ffl[l[1]] = True

In [234]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/FHierarchy_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/FHierarchy_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] in ffl and l[1] in ffl:
            fi = {'id': l[0], 'type': 'field', 'attr': l[2]}
            fj = {'id': l[1], 'type': 'field', 'attr': l[3]}
            graph.add_edge(fi, fj, relation_type = 'FF_in')
            ffl[l[0]] = fi
            ffl[l[1]] = fj

In [235]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PF_CS_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/PF_CS_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] in pfl and l[1] in ffl and type(ffl[l[1]]) == dict:
            pi = pfl[l[0]]
            fi = ffl[l[1]]
            graph.add_edge(pi, fi, time = pi['time'], relation_type = 'PF_in_' + fi['attr'])

In [237]:
coi = defaultdict(lambda: [])
coa = defaultdict(lambda: {})
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PAuAf_CS_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/PAuAf_CS_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] in pfl and l[2] in vfi_ids:
            pi = pfl[l[0]]
            ai = {'id': l[1], 'type': 'author'}
            fi = {'id': l[2], 'type': 'affiliation'}
            coa[l[0]][int(l[-1])] = ai
            coi[l[0]] += [fi]
            graph.add_edge(ai, fi, relation_type = 'in')
            pid = graph.node_forward['paper'][l[0]]
            for rel in graph.edge_list['paper']['venue']:
                for vid in graph.edge_list['paper']['venue'][rel][pid]:
                    graph.add_edge(ai, graph.node_bacward['venue'][vid], relation_type = 'APV_in')
            for rel in graph.edge_list['paper']['field']:
                for fid in graph.edge_list['paper']['field'][rel][pid]:
                    graph.add_edge(ai, graph.node_bacward['field'][fid], relation_type = 'APF_in')

In [238]:
for pid in tqdm(coa):
    pi = pfl[pid]
    max_seq = max(coa[pid].keys())
    for seq_i in coa[pid]:
        ai = coa[pid][seq_i]
        if seq_i == 1:
            graph.add_edge(ai, pi, time = pi['time'], relation_type = 'AP_write_first')
        elif seq_i == max_seq:
            graph.add_edge(ai, pi, time = pi['time'], relation_type = 'AP_write_last')
        else:
            graph.add_edge(ai, pi, time = pi['time'], relation_type = 'AP_write_other')
        for seq_j in coa[pid]:
            if seq_j > seq_i:
                aj = coa[pid][seq_j]
                graph.add_edge(ai, aj, time = pi['time'], relation_type = 'APA_coauthor', directed = False)

In [239]:
for pid in tqdm(coi):
    pi = pfl[pid]
    for seq_i, fi in enumerate(coi[pid]):
        for seq_j, fj in enumerate(coi[pid]):
            if seq_j > seq_i:
                graph.add_edge(fi, fj, time = pi['time'], relation_type = 'IPI_coauthor', directed = False)

In [240]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/vfi_vector.tsv', 'r'))
with open('/datadrive/MAG_0919/vfi_vector.tsv', 'r') as fin:
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        ser = l[0]
        for idx in ['venue', 'field', 'affiliation']:
            if ser in graph.node_forward[idx]:
                graph.node_bacward[idx][graph.node_forward[idx][ser]]['node_emb'] = np.array(l[1].split(' '))

In [241]:
for idx in ['venue', 'field', 'affiliation']:
    tot = 0
    for i in graph.node_bacward[idx]:
        if 'node_emb' in i:
            tot += 1
    print(tot, len(graph.node_bacward[idx]))

6934 6934
45717 45717
9079 9079


In [242]:
for target_type in graph.node_bacward:
    print(target_type, len(graph.node_bacward[target_type]))

paper 544244
venue 6934
field 45717
author 510189
affiliation 9079


In [243]:
for target_type in graph.edge_list:
    print(target_type)
    for source_type in graph.edge_list[target_type]:
        print('-' * 4 + source_type)
        for rel_type in graph.edge_list[target_type][source_type]:
            tot = 0
            for tid in graph.edge_list[target_type][source_type][rel_type]:
                tot += len(graph.edge_list[target_type][source_type][rel_type][tid])
            print('-' * 8 + rel_type, tot)

venue
----paper
--------PV_Conference 296775
--------PV_Journal 228062
--------PV_Repository 19216
--------PV_Patent 191
----author
--------APV_in 1169046
paper
----venue
--------rev_PV_Conference 296775
--------rev_PV_Journal 228062
--------rev_PV_Repository 19216
--------rev_PV_Patent 191
----paper
--------PP_cite 5796354
--------rev_PP_cite 5796354
----field
--------rev_PF_in_L0 544371
--------rev_PF_in_L3 866423
--------rev_PF_in_L1 1197205
--------rev_PF_in_L2 2337525
--------rev_PF_in_L5 202221
--------rev_PF_in_L4 303541
----author
--------AP_write_last 429392
--------AP_write_other 662167
--------AP_write_first 454913
field
----field
--------FF_in 262526
--------rev_FF_in 262526
----paper
--------PF_in_L0 544371
--------PF_in_L3 866423
--------PF_in_L1 1197205
--------PF_in_L2 2337525
--------PF_in_L5 202221
--------PF_in_L4 303541
----author
--------APF_in 10403670
affiliation
----author
--------in 612872
----affiliation
--------IPI_coauthor 330225
author
----affiliation
-----

In [345]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/SeqName_CS_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/SeqName_CS_20190919.tsv', 'r') as fin:
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        key = l[2]
        if key in ['conference', 'journal', 'repository', 'patent']:
            key = 'venue'
        if key == 'fos':
            key = 'field'
        if l[0] in graph.node_forward[key]:
            s = graph.node_bacward[key][graph.node_forward[key][l[0]]]
            s['name'] = l[1]

In [346]:
res = []
for idx, pi in enumerate(graph.node_bacward['paper']):
    pi['citation'] = len(graph.edge_list['paper']['paper']['PP_cite'][idx])
    res += [[pi['title'], pi['citation']]]
res.sort(key=lambda x: x[1], reverse = True)
res[:10]

[['very deep convolutional networks for large scale image recognition', 4631],
 ['histograms of oriented gradients for human detection', 3713],
 ['adam a method for stochastic optimization', 3646],
 ['imagenet classification with deep convolutional neural networks', 3455],
 ['mapreduce simplified data processing on large clusters', 3304],
 ['imagenet a large scale hierarchical image database', 3241],
 ['deep residual learning for image recognition', 2934],
 ['deep residual learning for image recognition', 2622],
 ['imagenet large scale visual recognition challenge', 2560],
 ['rich feature hierarchies for accurate object detection and semantic segmentation',
  2396]]

In [347]:
res = []
for idx, ai in enumerate(graph.node_bacward['author']):
    citation = 0
    for rel in graph.edge_list['author']['paper'].keys():
        for pid in graph.edge_list['author']['paper'][rel][idx]:
            citation += graph.node_bacward['paper'][pid]['citation']
    ai['citation'] = citation
    res += [[ai['name'], ai['citation']]]
res.sort(key=lambda x: x[1], reverse = True)
res[:10]

[['li feifei', 13286],
 ['geoffrey e hinton', 12875],
 ['andrew zisserman', 12454],
 ['jian sun', 12281],
 ['ross b girshick', 11999],
 ['scott shenker', 11881],
 ['christopher d manning', 11235],
 ['david e culler', 11109],
 ['andrew y ng', 11040],
 ['rajkumar buyya', 10366]]

In [348]:
res = []
for idx, fi in enumerate(graph.node_bacward['affiliation']):
    citation = 0
    for aid in graph.edge_list['affiliation']['author']['in'][idx]:
        citation += graph.node_bacward['author'][aid]['citation']
    fi['citation'] = citation
    res += [[fi['name'], fi['citation']]]
res.sort(key=lambda x: x[1], reverse = True)
res[:10]

[['microsoft', 1181476],
 ['university of california berkeley', 1000070],
 ['stanford university', 869939],
 ['carnegie mellon university', 858205],
 ['massachusetts institute of technology', 849884],
 ['ibm', 785791],
 ['google', 666682],
 ['university of illinois at urbana champaign', 575059],
 ['university of california', 477534],
 ['university of washington', 427362]]

In [349]:
res = []
for idx, vi in enumerate(graph.node_bacward['venue']):
    citation = 0
    for rel in graph.edge_list['venue']['paper'].keys():
        for pid in graph.edge_list['venue']['paper'][rel][idx]:
            citation += graph.node_bacward['paper'][pid]['citation']
    vi['citation'] = citation
    res += [[vi['name'], vi['citation'], vi['attr']]]
res.sort(key=lambda x: x[1], reverse = True)
res[:10]

[['CVPR', 151542, 'Conference'],
 ['lecture notes in computer science', 126897, 'Conference'],
 ['CHI', 87812, 'Conference'],
 ['arxiv computer vision and pattern recognition', 80557, 'Repository'],
 ['sigplan notices', 74173, 'Conference'],
 ['INFOCOM', 61041, 'Conference'],
 ['ieee transactions on pattern analysis and machine intelligence',
  60607,
  'Journal'],
 ['ICCV', 59503, 'Conference'],
 ['NeurIPS', 55138, 'Conference'],
 ['computer communication review', 51380, 'Conference']]

In [351]:
res = []
for idx, fi in enumerate(graph.node_bacward['field']):
    citation = 0
    for rel in graph.edge_list['field']['paper'].keys():
        for pid in graph.edge_list['field']['paper'][rel][idx]:
            citation += graph.node_bacward['paper'][pid]['citation']
    fi['citation'] = citation
    res += [[fi['name'], fi['citation'], fi['attr']]]
res.sort(key=lambda x: x[1], reverse = True)
res[:100]

[['Computer science', 5796354, 'L0'],
 ['Artificial intelligence', 1907214, 'L1'],
 ['Machine learning', 1092810, 'L1'],
 ['Distributed computing', 936494, 'L1'],
 ['Real-time computing', 932037, 'L1'],
 ['Data mining', 848932, 'L1'],
 ['Computer network', 819089, 'L1'],
 ['Pattern recognition', 770406, 'L1'],
 ['Computer vision', 701076, 'L1'],
 ['Theoretical computer science', 527403, 'L1'],
 ['Parallel computing', 396259, 'L1'],
 ['Computer security', 333644, 'L1'],
 ['Database', 311806, 'L1'],
 ['Architecture', 284175, 'L2'],
 ['Information retrieval', 280678, 'L1'],
 ['Natural language processing', 238116, 'L1'],
 ['Electronic engineering', 236191, 'L1'],
 ['Software', 222512, 'L2'],
 ['Multimedia', 220977, 'L1'],
 ['Scalability', 216968, 'L2'],
 ['Algorithm', 208528, 'L1'],
 ['Human–computer interaction', 207960, 'L1'],
 ['Artificial neural network', 204302, 'L2'],
 ['The Internet', 192492, 'L2'],
 ['Wireless sensor network', 188220, 'L2'],
 ['Knowledge management', 183620, 'L1']

In [353]:
graph.propagate_feature()

In [354]:
del graph.node_bacward

In [355]:
dill.dump(graph, open(data_dir + 'graph.pk', 'wb'))

In [ ]:
# type_dict = {'field': 0, 'venue' : 1, 'paper' : 2, 'author': 3, 'affiliation': 4}
# for target_type in layer_data:
#     for _id in layer_data[target_type]:
#         yi = layer_data[target_type][_id]
#         plt.scatter(type_dict[target_type], yi, color = cm.hot(type_dict[target_type] / len(type_dict)), s = 100)
# #         plt.annotate(_id, (type_dict[target_type] - 0.1, yi + 0.05), fontsize = 12)
# for target_type in sub_graph_list:
#     for source_type in sub_graph_list[target_type]:
#         for relation_type in sub_graph_list[target_type][source_type]:
#             for e in sub_graph_list[target_type][source_type][relation_type]:
#                 x1 = type_dict[target_type]
#                 x2 = type_dict[source_type]
#                 y1 = e[0]
#                 y2 = e[1]
#                 plt.plot([x1, x2], [y1, y2], \
#                     color = cm.hot((x1 + x2) / 2 / len(type_dict)))
# plt.xticks(list(type_dict.values()), list(type_dict.keys()))